In [5]:
import sys
import os

sys.path.insert(0, '/home/guany20/workspace/QCFramework/QCElemental')
sys.path.insert(0, '/home/guany20/workspace/QCFramework/QCEngine')

from qcengine.programs.biovia.define import define
from qcengine.programs.biovia.cosmoprep import cosmoprep
from qcengine.programs.biovia.dscf import dscf
from pathlib import Path
import qcelemental as qcel
import qcengine as qcng
from qcengine.util import execute

# define--cosmoprep--dscf

data_root = Path('data')
mol = qcel.models.Molecule.from_file(data_root/'CCC.xyz')
qc_spec = qcel.models.common_models.Model(
    method="b-p", basis="def-TZVP"
)
qc_task = qcel.models.AtomicInput(
    molecule=mol, driver='optimization', model=qc_spec,
)



_, _, infiles = define(qc_task)
_, _, infiles = cosmoprep(infiles)

config=qcng.config.TaskConfig(
        **{"memory": 20, "nnodes": 1, "retries": 0, "scratch_directory": '/localscratch'}
)
exe_success, proc = dscf(infiles, config)

In [6]:
print(proc['stdout'])


                license information 
 Customer : 200000000282784
 ID       : LKO1837819

 License file is VALID, continuing with full version

   OpenMP run-time library returned nthreads =  8

 dscf (c001) : TURBOMOLE rev. V7.5.1 compiled 23 Dec 2020 at 11:16:29
 Copyright (C) 2020 TURBOMOLE GmbH, Karlsruhe


    2022-05-03 14:27:03.475 



                             d s c f - program

                   idea & directorship : reinhart ahlrichs
                   program development : marco haeser
                                         michael baer
                           dft version : oliver treutler


                          quantum chemistry group
                          universitaet  karlsruhe
                                  germany



   *-----------------------------------------------------------------------*
   |                     program will use  8 thread(s)                     |
   *-----------------------------------------------------------------------*


   

In [4]:
print(proc['stdout'])


                license information 
 Customer : 200000000282784
 ID       : LKO1837819

 License file is VALID, continuing with full version

   OpenMP run-time library returned nthreads =  8

 dscf (c001) : TURBOMOLE rev. V7.5.1 compiled 23 Dec 2020 at 11:16:29
 Copyright (C) 2020 TURBOMOLE GmbH, Karlsruhe


    2022-05-03 14:19:58.058 



                             d s c f - program

                   idea & directorship : reinhart ahlrichs
                   program development : marco haeser
                                         michael baer
                           dft version : oliver treutler


                          quantum chemistry group
                          universitaet  karlsruhe
                                  germany



   *-----------------------------------------------------------------------*
   |                     program will use  8 thread(s)                     |
   *-----------------------------------------------------------------------*


   

In [2]:
# COSMOTHERM

from qcengine.programs.biovia.cosmotherm import cosmotherm

cosmotherm_spec = qcel.models.common_models.Model(
    method="BP_TZVP",
    extra={
        'db': 'BP-TZVP-COSMO',
        'solvents_set': 'class23',
        'solvents_gibbs_id': 201
    }
)

exe_success, proc = cosmotherm(cosmotherm_spec, infiles=proc['outfiles'])

In [7]:
class23_solvent_ids = [0, 127, 152, 153, 154, 174, 176, 177, 178, 179, 183, 198, 201, 202, 238, 241, 244, 269, 363, 6573, 372, 379, 394, 818, 1137, 1176, 1183, 1226, 1227, 1229, 1256, 1260, 1276, 1280, 1285, 1299, 1312, 1326, 1334, 1336, 1339, 1515, 1552, 1617, 1630, 1632, 1663, 1677, 1771, 1791, 2875, 3925, 4489]
import sys
import os

sys.path.insert(0, '/home/guany20/workspace/QCFramework/QCElemental')
sys.path.insert(0, '/home/guany20/workspace/QCFramework/QCEngine')

from qcengine.programs.biovia.define import define
from qcengine.programs.biovia.cosmoprep import cosmoprep
from qcengine.programs.biovia.dscf import dscf
from pathlib import Path
import qcelemental as qcel
import qcengine as qcng
from qcengine.util import execute
from qcengine.programs.biovia.cosmotherm import cosmotherm

xyzs = [Path('data/xyzs')/x for x in os.listdir(Path('data/xyzs'))]
for xyz in xyzs:
    mol = qcel.models.Molecule.from_file(xyz)
    qc_spec = qcel.models.common_models.Model(
        method="b-p", basis="def-TZVP"
    )
    qc_task = qcel.models.AtomicInput(
        molecule=mol, driver='optimization', model=qc_spec,
    )



    _, _, infiles = define(qc_task)
    _, _, infiles = cosmoprep(infiles)

    config=qcng.config.TaskConfig(
            **{"memory": 20, "ncores": 8, "nnodes": 1, "retries": 0, "scratch_directory": '/localscratch'}
    )
    exe_success, proc = dscf(infiles, config)
    
    
    for cosmo_id in class23_solvent_ids:
        cosmotherm_spec = qcel.models.common_models.Model(
            method="BP_TZVP",
            extra={
                'db': 'BP-TZVP-COSMO',
                'solvents_set': 'class23',
                'solvents_gibbs_id': 201
            }
        )

        exe_success, proc_cosmo = cosmotherm(cosmotherm_spec, infiles=proc['outfiles'])
    
        name = xyz.name.split('.')[0]
        os.makedirs(f'output/{name}/{cosmo_id}', exist_ok=True)
        with open(f'output/{name}/{cosmo_id}/cosmo.tab', 'w') as fh:
            fh.write(proc_cosmo['outfiles']['cosmotherm.tab'])


KeyboardInterrupt

